### ------------------------------------------------------------------------------------------------------------

In [ ]:
import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/impulsecorp/PickStocks.git
    !mv PickStocks/*.py .
    !mv PickStocks/data .
    !pip install -U -qq -r PickStocks/requirements.txt

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import system
from system import *
# small hack to prevent Colab error
try:
    from datablock import *
except:
    from datablock import *
seed

In [ ]:
%pylab inline

### Setup

In [ ]:
data_timeperiod = 'D'
data = get_data('SPY', period=data_timeperiod, nrows=None)
data = procdata_lite(data)

In [ ]:
# for inspectiion
print(data.shape)
data.head()

In [ ]:
system.train_set_end = 0.7 # percentage point specifying the training set end point (1.0 means all data is training set)
system.val_set_end = 1.0    # percentage point specifying the validation set end point (1.0 means no test set)
# basically this is the data with the values above, which are like sliders determining the layout
# [|0.0| ................... train .............. |0.7| ....................... val ..................... |1.0| no test ]
system.balance_data = 1

### ------------------------------------------------------------------------------------------------------------

### LogisticRegression

In [ ]:
# Train LogisticRegression classifier on train data
clf, scaler = train_classifier(LogisticRegression, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### LogisticRegression + HPO

In [ ]:
# Train LogisticRegression classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(LogisticRegression, 'LogisticRegression', {"C": hp.loguniform("C", -5, 2),
                                                                               "max_iter": hp.choice("max_iter", range(5, 501)),
                                                                               "dual": hp.choice("dual", (True, False)),
                                                                               "fit_intercept": hp.choice("fit_intercept", (True, False))},
                                  X_train, y_train, max_evals=100)
clf, scaler = train_classifier(LogisticRegression, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### XGBClassifier

In [ ]:
# Train XGBClassifier classifier on train data
clf, scaler = train_classifier(XGBClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### XGBClassifier + HPO

In [ ]:
# Train XGBClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(XGBClassifier, 'XGBClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "learning_rate": hp.loguniform("learning_rate", -5, 0), "max_depth": hp.choice("max_depth", range(2, 11))},
                                  X_train, y_train, max_evals=10)
clf, scaler = train_classifier(XGBClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### LGBMClassifier

In [ ]:
# Train LGBMClassifier classifier on train data
clf, scaler = train_classifier(LGBMClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### LGBMClassifier + HPO

In [ ]:
# Train LGBMClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(LGBMClassifier, 'LGBMClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                  "learning_rate": hp.loguniform("learning_rate", -5, 0),
                                  "max_depth": hp.choice("max_depth", range(2, 11))},
                                  X_train, y_train, max_evals=10)
clf, scaler = train_classifier(LGBMClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### RandomForestClassifier

In [ ]:
# Train RandomForestClassifier classifier on train data
clf, scaler = train_classifier(RandomForestClassifier, data, n_jobs=-1)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### RandomForestClassifier + HPO

In [ ]:
# Train RandomForestClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(RandomForestClassifier, 'RandomForestClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "max_depth": hp.choice("max_depth", range(2, 21))},
                                  X_train, y_train, max_evals=10,
                                  n_jobs=-1)
clf, scaler = train_classifier(RandomForestClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### ExtraTreesClassifier

In [ ]:
# Train ExtraTreesClassifier classifier on train data
clf, scaler = train_classifier(ExtraTreesClassifier, data)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ExtraTreesClassifier + HPO

In [ ]:
# Train ExtraTreesClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(ExtraTreesClassifier, 'ExtraTreesClassifier',
                                  {"n_estimators": hp.choice("n_estimators", range(5, 201)),
                                   "max_depth": hp.choice("max_depth", range(2, 21))},
                                  X_train, y_train, max_evals=10,
                                  n_jobs=-1)
clf, scaler = train_classifier(ExtraTreesClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------

### CatBoostClassifier

In [ ]:
# Train CatBoostClassifier classifier on train data
clf, scaler = train_classifier(CatBoostClassifier, data, verbose=False)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### CatBoostClassifier + HPO

In [ ]:
# Train CatBoostClassifier classifier on train data, but optimize it with HPO first

X_train, y_train = get_clean_Xy(data.iloc[0:int(data.shape[0] * system.train_set_end)])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
if system.balance_data:
    # Apply SMOTE oversampling to balance the training data
    sm = SMOTE(random_state=newseed())
    X_train, y_train = sm.fit_resample(X_train, y_train)
best_hyperparams = optimize_model(CatBoostClassifier, 'CatBoostClassifier',
                                  {
                                    "iterations": hp.quniform("iterations", 50, 1200, 1),
                                    "learning_rate": hp.uniform("learning_rate", 0.01, 0.2),
                                    "depth": hp.quniform("depth", 4, 10, 1),
                                    "l2_leaf_reg": hp.uniform("l2_leaf_reg", 1, 10),
                                    "border_count": hp.quniform("border_count", 128, 256, 1),
                                    "random_strength": hp.uniform("random_strength", 0.5, 2),
                                    "bagging_temperature": hp.uniform("bagging_temperature", 0.5, 2),
                                },
                                  X_train, y_train, max_evals=10, verbose=False)
best_hyperparams['verbose'] = False
clf, scaler = train_classifier(CatBoostClassifier, data, **best_hyperparams)

In [ ]:
# Test on val data
equity, pf, trades = qbacktest(clf, scaler, data)

In [ ]:
trades.head()

### ------------------------------------------------------------------------------------------------------------